### Enter variables

In [1]:
# Input files
storage_shapefile = "BR_Sample_Parcels.shp"
storage_id_field = "qldglobe_p"

# Output files
outputCSV = "GEE_Map_Cropping_Landsat.csv"

# Cloud cover
CLOUD_FILTER = 20 # Under 40%

# Thresholds
ndwiMaskVal = -0.1
ndviMaskVal = -0.1

# UTM Zone
utmZoneInfo = 'EPSG:28355'#z55S

# Dates
start_date = '1986-01-01'
end_date = '2023-01-01'

### Import modules

In [2]:
import ee, geemap, folium, pandas, geopandas as gpd, json, datetime, altair as alt, csv
from IPython.display import Image
import ee.mapclient
import datetime
import os
from os.path import exists as file_exists

import requests

# importing pandas as pd
import pandas as pd

### Declare functions

In [3]:
def test_api():
    dem = ee.Image('USGS/SRTMGL1_003')
    xy = ee.Geometry.Point([86.9250, 27.9881])
    elev = dem.sample(xy, 30).first().get('elevation').getInfo()
    print('Authentication active')

#NDVI calculation:
def areadate(img):
    area = img.gt(0).multiply(ee.Image.pixelArea()).divide(1000000).reduceRegion(ee.Reducer.sum(), storage_geometry, 30).get('NDWI');
    return img.set('area', area).set('date', img.get('system:time_start'));

def ndwi_func(i):
    ndwi = i.normalizedDifference (['B2', 'B5']).rename('NDWI')
    return i.addBands(ee.Image(ndwi))

def addDate(image):
    # parse date stored in 'system:index'
    date = ee.Date(image.get('system:index'))

    # format date, see http:#www.joda.org/joda-time/apidocs/org/joda/time/format/DateTimeFormat.html
    str = date.format('YYYY-mm-dd')

    return image.set({'Date': str})

def mean_func(image):
    mean = (image                        
            #.mean()
            .reduceRegion(ee.Reducer.mean()))#, storage_geometry, 30))
    mean = mean.getInfo()            
    mean = mean.get('nd')    
    return mean

def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (l57ndwi
            .filter(range)
            .select('NDWI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), storage_geometry, 30))
    return mean.set('year', y)

def get_area(img):
    total_area = img.multiply(ee.Image.pixelArea())
    ndwi = img.gte(ndwiMaskVal).rename('ndwiMaskLayer')
    ndwi_area = ndwi.multiply(ee.Image.pixelArea())
    return img.set('total_area', total_area).set('ndwi_area', ndwi_area)

def test_api():
    dem = ee.Image('USGS/SRTMGL1_003')
    xy = ee.Geometry.Point([86.9250, 27.9881])
    elev = dem.sample(xy, 30).first().get('elevation').getInfo()
    print('Authentication active')
    
def transformer(feature):
    transformed_feature = feature.transform(utmZoneInfo, 0.001);
    return transformed_feature;

#def get_area(feature):
#    area = feature.area()
#    area = ee.Number(area).divide(1e6).round()
#    return area

def extract_water(img):
    ndwi_image = img.normalizedDifference(['B2', 'B5'])
    water_image = ndwi_image.gt(ndwiMaskVal)
    return water_image

def get_stats(image):
    date = ee.Date(image.get('system:time_start'))        
    date = date.format('dd-MM-YYYY')
    allPixels = image.get('allPixels')
    allPixels = allPixels.getInfo()
    ndwiPixels = image.get('ndwiPixels')
    ndwiPixels = ndwiPixels.getInfo()
    CLOUD_COVER = image.get('CLOUD_COVER')
    CLOUD_COVER = CLOUD_COVER.getInfo()
    return date, allPixels, ndwiPixels, CLOUD_COVER

def filterImages(storage_geometry):
    # Load Landsat 5 & 7
    l5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
    l7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
    # Merge Landsat 5 & 7
    l57merge = l5.merge(l7) 
    # Filter landsat collections by area, cloud cover and bands required for NDWI
    l57filter = (l57merge.filterBounds(storage_geometry)
              .filterDate(start_date, end_date)
              .select(['B1','B2','B3','B4','B5'])
              .map(lambda image: image.clip(storage_geometry).reproject(crs=utmZoneInfo)))
              #.filterMetadata('CLOUD_COVER', 'less_than', CLOUD_FILTER))
    # Add date
    l57filter.map(addDate)
    # Sort by date
    images = l57filter.sort('Date')
    return images

def exportStats(worksNumber):
    # Create a new dataframe
    storageStats = pandas.DataFrame(columns=['Works Number', 'Date', 'Total Area', 'Water Area', 'NDVI', 'NDWI', 'MNDWI', 'Cloud Cover'])        
    # Select the feature
    storage_selection = storage_collection.filter(ee.Filter.eq("Wks_Num_tx", worksNumber))   
    # Get the geometry
    storage_geometry = storage_selection.geometry()
    # Create list of images
    images = filterImages(storage_geometry)
    # Create a list of images
    imageSize = images.size().getInfo()    
    imageList = images.toList(images.size())            
    # Get stats ##
    # For each image
    for i in range(0, imageSize):    
        # Choose the image
        image = ee.Image(imageList.get(i))
        # Get date
        date = ee.Date(image.get('system:time_start')).format("yyyy-MM-dd")
        date = date.getInfo()
        # Create images
        ndvi_image = image.normalizedDifference(['B4', 'B3'])
        ndwi_image = image.normalizedDifference(['B4', 'B5'])
        mndwi_image = image.normalizedDifference(['B2', 'B5'])
        # Get the area of the storage
        storage_area = storage_geometry.area(1).getInfo()
        # Get water area - Mask water areas using MNDWI
        water_image = mndwi_image.gt(ndwiMaskVal).selfMask()
        water_vector = water_image.reduceToVectors()
        water_area = water_vector.geometry().area(1).getInfo()
        # Get average ndvi, ndwi and mndwi values            
        ndvi = mean_func(ndvi_image)
        ndwi = mean_func(ndwi_image)
        mndwi = mean_func(mndwi_image)
        # Get cloud cover
        cloudiness = image.get("CLOUD_COVER").getInfo()
        # Add results to storage stats dataframe
        results = [worksNumber,date,storage_area,water_area,ndvi,ndwi,mndwi,cloudiness]
        #print(results)
        storageStats.loc[len(storageStats)] = results   
        # Export image
        #exportRGBImage(image)
    # Save to CSV
    storageStats.to_csv(outputCSV)
    
def exportRGBImage(image):
    # Create a image filename
    filename = worksNumber + "_" + str(date) + ".tif"
    # Create a RGB image for export
    stats = image.reduceRegion(reducer=ee.Reducer.minMax(),geometry=storage_geometry,scale=100,bestEffort=True)
    statDict = stats.getInfo()
    prettyImg = image.visualize(bands=['B3', 'B2', 'B1'],
             min=[statDict['B3_min'],statDict['B2_min'],statDict['B1_min']],
             max=[statDict['B3_max'],statDict['B2_max'],statDict['B1_max']],
             gamma=1)
    # Export multi-band GeoTIFF file.
    url = prettyImg.getDownloadUrl({
        #'bands': ['B3', 'B2', 'B1'],
        #'region': region,
        #'scale': 20,
        'format': 'GEO_TIFF'
    })            
    response = requests.get(url)
    with open(filename, 'wb') as fd:                
      fd.write(response.content)
    return

### Authenticate Earth Engine

In [4]:
# Test access then authenticate if fails
try:
    test_api()
except:    
    ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AWtgzh4a2dtwps6LZYD5XbnCUJ6ecwnC-5539a6NiyDlRf7EFMk7MkbSWug

Successfully saved authorization token.


### Convert shapefile into feature collection

In [5]:
# Load shapefile into geopandas
storage_shapes = gpd.read_file(storage_shapefile)

# Convert shapes into json
json_storages = json.loads(storage_shapes.to_json())

# Convert json into feature collection
storage_collection =  ee.FeatureCollection(json_storages)

# Reproject collection
storage_collection = storage_collection.map(transformer);

# Add area
storage_collection.map(lambda feature: feature.set('total_area', ee.Number(feature.area()).divide(1e6).round()))

### Get a list of Storage IDs

In [6]:
# list the unique quality code values
worksSet = sorted(storage_shapes[storage_id_field].unique())
print(worksSet)

['101SP207166', '108SP207166', '110F6901', '111F6901', '11RP218867', '14CVN407', '15CVN386', '16BLM76', '17CVN407', '1BLM66', '1BLM846', '1PER4265', '1RP225469', '1SP132773', '236RP851344', '23SP225448', '24PG406', '25PG260', '297SP146073', '2BLM1', '2BLM465', '2SP276749', '31BLM809', '33CP849174', '39BLM782', '3BLM1', '3RP225469', '3SP276749', '4BEL5359', '4SP276749', '5RP212965', '5RP213579', '5SP225462', '5SP276749', '612RP909550', '62SP225462', '63CP858891', '6RP213579', '71RP213572', '73RP213572', '7BLM715', '8CVN140', 'AAP21199']


### Clip imagery to each storage

In [ ]:
# Start timer
startNow = datetime.datetime.now()
print("Starting at " + str(startNow))

# For each storage
for worksNumber in worksSet:
    
    # Create file name for csv export
    outputCSV = worksNumber + "_Landsat_Timeseries.csv"
    
    # Check if already completed
    if file_exists(outputCSV):
        
        print("Skipping " + x)
        
    else:
        
        # Create an image function to map
        exportStats(worksNumber)
        
# Print the storage stats
print("Finished")

# Finish time
endNow = datetime.datetime.now()
diff = (endNow-startNow).total_seconds()
print("End at " + str(endNow))
print("Taking: " + str(diff) + " seconds")

Starting at 2022-09-09 08:06:03.792559
